# 1. Create a New Notebook & Import Everything

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Define project path
path = r'C:\Users\Jacques\OneDrive\Documents\Data Analytics course\Data Immersion\Section 4\08 April 2025 Instacart Basket Analysis'

# Import the full merged dataset with previous columns included
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_derived.pkl'))

In [4]:
# Check the data structure
ords_prods_merge.shape
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,product_merge_flag,price_label,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,prior,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regular,Average orders
1,2539329,1,prior,1,2,8,NaN,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regular,Average orders
2,2539329,1,prior,1,2,8,NaN,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Regular,Average orders
3,2539329,1,prior,1,2,8,NaN,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Regular,Average orders
4,2539329,1,prior,1,2,8,NaN,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Regular,Average orders


# 2. Aggregate Mean of 'order_number' by 'department_id' (full dataset)

In [16]:
# Aggregates the mean of 'order_number' by 'department_id'
dept_order_mean = ords_prods_merge.groupby('department_id').agg({'order_number': 'mean'})
print(dept_order_mean)

               order_number
department_id              
1                 14.800024
2                 17.091743
3                 17.913544
4                 17.893092
5                 15.214270
6                 15.382135
7                 17.694027
8                 16.458105
9                 15.957363
10                20.091818
11                16.482026
12                15.615061
13                16.484023
14                17.524632
15                15.691875
16                18.014071
17                16.150593
18                19.602850
19                17.631340
20                17.138607
21                21.956893


# Markdown Analysis
# Compared to the subset, the average number of orders per department in the full dataset reflects more accurate global patterns. 
# For example, while Produce still has the highest average, the values have slightly shifted due to more data being included.`

# 3. Create Loyalty Flag (Repeat Transform + loc Workflow)

In [22]:
# Get the max orders per user
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

C:\Users\Jacques\AppData\Local\Temp\ipykernel_12440\1582167655.py:2: FutureWarning: The provided callable <function max at 0x0000028AF9A4E7A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


In [24]:
# Assign loyalty levels using loc():
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# 4. Compare Spending by Loyalty Category

In [29]:
# Get basic descriptive statistics for the different types of customers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'median']})

prices                     
                       mean  min      max median
loyalty_flag                                    
Loyal customer     9.081626  1.0  99999.0    7.3
New customer      15.843383  1.0  99999.0    7.4
Regular customer  11.139256  1.0  99999.0    7.4

# Markdown Analysis

# From the summary statistics, we observe the following:

# New customers have the highest average price per product purchased (`15.84`), suggesting that they may be buying fewer but more expensive items on their first few orders.
# Loyal customers have the lowest average price (`9.08`), which may indicate more frequent purchases of lower-cost everyday items, possibly due to established shopping habits.
# Regular customers fall in between, with an average price of `11.14`.

# Interestingly, the median prices are fairly consistent across all segments (around `7.3–7.4`), indicating that the majority of items purchased fall within a similar price range,
# but the mean is skewed by very high-priced outliers (as shown by the `max` value of `99999.0` across all groups). This suggests there are likely some data quality issues or anomalies that should be cleaned or filtered in future analyses.

# 5. Create Spending Flag Based on Average Price per User

In [34]:
# Calculate average price per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

In [36]:
# Create the flag
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# 6. Create Order Frequency Flag

In [39]:
# Calculate median days since prior order
ords_prods_merge['median_days'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

In [41]:
# Create flag based on median
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'order_freq_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'order_freq_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'order_freq_flag'] = 'Frequent customer'

# 7. Export the Updated DataFrame

In [44]:
# Export the Updated DataFrame
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_enriched.pkl'))